In [ ]:
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from test_setup import problems


def plot_pf_d2(problem_name, problem, seed_ind_data):
    true_pf = problem._calc_pareto_front().T
    fig1 = px.line(x=true_pf[0], y=true_pf[1])
    fig1.update_traces(
        line=dict(color = 'rgba(50,50,50,0.4)'))

    #seed_ind_data = seed_ind_data[seed_ind_data.problem_name == problem_name]
    d2_seed_types_to_plot = ["e1", "e2", "c"]
    seed_ind_data = seed_ind_data[seed_ind_data.type.isin(d2_seed_types_to_plot)]
    fig2 = px.scatter(seed_ind_data, x="obj_1", y="obj_2", color="type")
    fig2.update_traces(marker_size = 9)

    fig3 = go.Figure(data=fig1.data + fig2.data)
    fig3.update_layout(
        autosize=False,
        margin=dict(
            l=0,  # left margin
            r=0,  # right margin
            b=0,  # bottom margin
            t=40,  # top margin
            pad=0  # padding
        ),
        height=300,
        width=375, #to make d2 and d3 the same size
        title=problem_name,
        xaxis_title="f1",
        yaxis_title="f2",
        font_size=12,
    )

    pareto_set_F = problem._calc_pareto_front(n_pareto_points=3)
    upper_bounds = pareto_set_F.max(axis=0) 
    lower_bounds = pareto_set_F.min(axis=0)

    #center
    fig3.add_shape(
            type="line",
            x0=lower_bounds[0],
            y0=lower_bounds[1],
            x1=upper_bounds[0]*0.39, #0.39 to make the vector cut off at the right point
            y1=upper_bounds[1]*0.39,
            line=dict(
                color=px.colors.qualitative.Plotly[2],
                width=4,
                dash="dot",
            ),
            label=dict(text="vector c")
    )
    print(px.colors.qualitative.Plotly)
    #edge 2
    fig3.add_shape(
            type="line",
            x0=lower_bounds[0],
            y0=lower_bounds[1],
            x1=upper_bounds[0],
            y1=lower_bounds[1],
            line=dict(
                color=px.colors.qualitative.Plotly[1],
                width=4,
                dash="dot",
            ),
            label=dict(text="vector e2")
    )

    #edge 1
    fig3.add_shape(
            type="line",
            x0=lower_bounds[0],
            y0=lower_bounds[1],
            x1=lower_bounds[0],
            y1=upper_bounds[1],
            line=dict(
                color=px.colors.qualitative.Plotly[0],
                width=4,
                dash="dot",
            ),
            label=dict(text="vector e1")
    )

    fig3.write_image("../figures/paper/seed_ind_generation_"+problem_name+".pdf")
    fig3.show()


def plot_pf_d3(problem_name, problem, seed_ind_data):
    true_pf = problem._calc_pareto_front(n_pareto_points=625).T
    fig1 = px.scatter_3d(x=true_pf[0], y=true_pf[1], z=true_pf[2])
    fig1.update_traces(marker=dict(
        color = 'rgba(50,50,50,0.4)'),
        marker_size = 6)

    #seed_ind_data = seed_ind_data[seed_ind_data.problem_name == problem_name]
    d3_seed_types_to_plot = ["e1", "e2", "e3", "c"]
    seed_ind_data = seed_ind_data[seed_ind_data.type.isin(d3_seed_types_to_plot)]
    fig2 = px.scatter_3d(seed_ind_data, x="obj_1", y="obj_2", z="obj_3", color="type")


    name = 'eye = (x:2, y:2, z:2)'
    camera = dict(
        eye=dict(x=1.1, y=1.1, z=1.6)
    )

    fig3 = go.Figure(data=fig2.data + fig1.data)
    fig3.update_layout(
        scene_camera=camera,
        autosize=False,
        margin=dict(
            l=0,  # left margin
            r=0,  # right margin
            b=00,  # bottom margin
            t=40,  # top margin
            pad=0  # padding
        ),
        height=400,
        width=500,
        title=problem_name,
        font_size=16,
        scene=dict(
            xaxis_title="f1",
            yaxis_title="f2",
            zaxis_title="f3",
        ),
    )
    fig3.write_image("../figures/paper/seed_ind_generation_"+problem_name+".pdf")
    fig3.show()


for problem_name in problems.keys():
    problem = problems[problem_name]
    fitness_data = pd.read_csv("../data/seed_individuals/seed_individuals_"+problem_name+".csv")
    if problem_name == "UF1": #problem.n_obj == 2: #only plot Maco and DTLZ1
        plot_pf_d2(problem_name, problem, fitness_data)
    if problem_name == "UF8": #problem.n_obj == 3:
        plot_pf_d3(problem_name, problem, fitness_data)

# Build Performance Table

In [ ]:
import pandas as pd
baseline_comparison_d2 = pd.read_csv("../data/e_and_c_benchmark/baseline_comparison_data_d2.csv.tar.gz")
baseline_comparison_d2 = baseline_comparison_d2.loc[baseline_comparison_d2.generation == 100]
baseline_comparison_d2 = baseline_comparison_d2.loc[(baseline_comparison_d2.problem_name != "ZDT4") & (baseline_comparison_d2.problem_name != "ZDT6") & (baseline_comparison_d2.problem_name != "MACO_w=shallow")]#filter unused problems
baseline_comparison_d3 = pd.read_csv("../data/e_and_c_benchmark/baseline_comparison_data_d3.csv.tar.gz")
baseline_comparison_d3 = baseline_comparison_d3.loc[baseline_comparison_d3.generation == 100]

baseline_comparison_d2

In [17]:
def get_partial_table(performance_indicator, algorithm, dataframe):
    current_df = dataframe.loc[dataframe.algorithm == algorithm]
    results_df = pd.DataFrame(columns=current_df.problem_name.unique(), index=current_df.seed_type.unique(), dtype=str)
    for problem_name in current_df.problem_name.unique():
        for seed_type in current_df.seed_type.unique():
            results_df[problem_name][seed_type] = current_df.loc[(current_df.problem_name == problem_name) & (current_df.seed_type == seed_type)].iloc[0][performance_indicator]
    return results_df

def build_table(data, dim):
    performance_indicators = ["average_igd+_text", "average_hv_text"]
    algorithms=data.algorithm.unique()

    single_solution_performance_indicator = []
    for performance_indicator in performance_indicators:
        single_res_palgorithm = []
        for algorithm in algorithms:
            single_res_palgorithm.append( get_partial_table(performance_indicator, algorithm, data) )
        results_df = pd.concat(single_res_palgorithm, keys=algorithms)
        single_solution_performance_indicator.append(results_df)
    df_concat = pd.concat(single_solution_performance_indicator,axis=1,keys=performance_indicators)

    latex_string = df_concat.to_latex(multicolumn=True, multirow=True, escape=False)
    latex_string = latex_string.replace("\\multirow[t]", "\\multirow[c]")#center the multirow
    latex_string = latex_string.replace("{r}", "{c}")#center the multicol
    latex_string = latex_string.replace("_text", "")#remove this lable from the performance indicator
    latex_string = latex_string.replace("_", " ")#center the multirow
    latex_string = latex_string.replace("llllllllllllllllllll", "p{0.44cm}p{0.44cm}|p{0.44cm}p{0.44cm}p{0.44cm}p{0.44cm}p{0.44cm}p{0.44cm}p{0.44cm}p{0.44cm}p{0.44cm}||p{0.44cm}p{0.44cm}p{0.44cm}p{0.44cm}p{0.44cm}p{0.44cm}p{0.44cm}p{0.44cm}p{0.44cm}")#center the multirow d2
    latex_string = latex_string.replace("llllllllllllll", "p{0.5cm}p{0.8cm}|p{0.5cm}p{0.5cm}p{0.5cm}p{0.5cm}p{0.5cm}p{0.5cm}||p{0.5cm}p{0.5cm}p{0.66cm}p{0.5cm}p{0.5cm}p{0.8cm}")#center the multirow d3
    with open("../figures/paper/performance_comparison/performance_comparison_table_d"+str(dim)+".tex", "w") as text_file:
        text_file.write(latex_string)

table_d2_df = baseline_comparison_d2[["problem_name", "algorithm", "seed_type", "average_igd+_text", "percentual_igd+_diff_text", "average_hv_text", "percentual_hv_diff_text"]]
table_d3_df = baseline_comparison_d3[["problem_name", "algorithm", "seed_type", "average_igd+_text", "percentual_igd+_diff_text", "average_hv_text", "percentual_hv_diff_text"]]

build_table(table_d2_df, 2)
build_table(table_d3_df, 3)

In [4]:
# Heiner Gedächnis Table (unused)

table_d2_df = baseline_comparison_d2[["problem_name", "algorithm", "seed_type", "average_igd+", "percentual_igd+_diff_text", "average_hv", "percentual_hv_diff_text"]]

# post-process the latex string
latex_string = table_d2_df.to_latex(index=False, sparsify=True, multicolumn=True, multirow=True, escape=False)
latex_string = latex_string.replace("\\multirow[t]", "\\multirow[c]")#center the multirow
latex_string = latex_string.replace("{r}", "{c}")#center the multicol
#latex_string = latex_string.replace("best", "b")#shorten the row names
#latex_string = latex_string.replace("lower quartile", "lq")
latex_string = latex_string.replace("seed_type", "seed type")
latex_string = latex_string.replace("average_igd+", "igd+")
latex_string = latex_string.replace("percentual_igd+_diff_text", "igd+ diff")
latex_string = latex_string.replace("average_hv", "hv")
latex_string = latex_string.replace("percentual_hv_diff_text", "hv diff")
latex_string = latex_string.replace("problem_name", "problem")
latex_string = latex_string.replace("_", "\_") #for the MACO_b
latex_string = latex_string.replace("lllrlrl", "ccc||cc||cc")# insert hlines

#write the file
with open("../figures/paper/performance_comparison/performance_comparison_table_d2.tex", "w") as text_file:
    text_file.write(latex_string)



table_d3_df = baseline_comparison_d3[["problem_name", "algorithm", "seed_type", "average_igd+", "percentual_igd+_diff_text", "average_hv", "percentual_hv_diff_text"]]

# post-process the latex string
latex_string = table_d3_df.to_latex(index=False, sparsify=True, multicolumn=True, multirow=True, escape=False)
latex_string = latex_string.replace("seed_type", "seed type")
latex_string = latex_string.replace("average_igd+", "igd+")
latex_string = latex_string.replace("percentual_igd+_diff_text", "igd+ diff")
latex_string = latex_string.replace("average_hv", "hv")
latex_string = latex_string.replace("percentual_hv_diff_text", "hv diff")
latex_string = latex_string.replace("problem_name", "problem")
latex_string = latex_string.replace("_", "\_") #for the MACO_b
latex_string = latex_string.replace("lllrlrl", "ccc||cc||cc")# insert hlines

with open("../figures/paper/performance_comparison/performance_comparison_table_d3.tex", "w") as text_file:
    text_file.write(latex_string)

# Comparing with Random as Baseline / Average Performance Indicator Values

In [18]:
import pandas as pd
baseline_comparison_d2 = pd.read_csv("../data/e_and_c_benchmark/baseline_comparison_data_d2.csv")
baseline_comparison_d2 = baseline_comparison_d2.loc[baseline_comparison_d2.generation == 100]
baseline_comparison_d2 = baseline_comparison_d2.loc[(baseline_comparison_d2.problem_name != "ZDT4") & (baseline_comparison_d2.problem_name != "ZDT6") & (baseline_comparison_d2.problem_name != "MACO_w=shallow")]#filter unused problems
baseline_comparison_d3 = pd.read_csv("../data/e_and_c_benchmark/baseline_comparison_data_d3.csv")
baseline_comparison_d3 = baseline_comparison_d3.loc[baseline_comparison_d3.generation == 100]


In [ ]:
# Heatmaps of the percentual difference
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


def plot_heatmap(dataframe, indicator_name, dim):
    algorithm_names = dataframe.algorithm.unique()
    fig = make_subplots(cols=1, rows=len(algorithm_names), shared_xaxes=True, x_title=indicator_name,
                        horizontal_spacing = 0.001,
                        vertical_spacing = 0.066,
                        row_titles=["NSGA-II", "MOEA/D"])#, row_titles=algorithm_names, start_cell="top-left")

    dataframe = dataframe.replace(to_replace="e1+e2",value="e1+2") #rename the seed types for better readability
    dataframe = dataframe.replace(to_replace="e1+e2+e3",value="e1+2+3")
    dataframe = dataframe.replace(to_replace="e1+e2+c",value="e1+2+c")
    dataframe = dataframe.replace(to_replace="e1+e2+e3+c",value="e1+2+3+c")
    for i, algorithm_name in enumerate(algorithm_names):
        plot_data = dataframe.loc[(dataframe['seed_type'] != 'r') & (dataframe['algorithm'] == algorithm_name)]
        fig.add_trace(go.Heatmap(
            z=plot_data[indicator_name],
            zmid=0.0,
            zmin=-1.0,
            zmax=1.0,
            x=plot_data['problem_name'],
            y=plot_data['seed_type'],
            text=plot_data[indicator_name+"_text"],
            texttemplate="%{text}",
            textfont={"size":11},
            colorscale='RdBu',
            colorbar=dict(title="Relative<br>IGD+ Diff")),
            col=1, row=i+1,
        )
        # Update x-axis and y-axis labels
    if dim == 2:#this is bodged, but only update the 2D MACO problem names
        fig.update_xaxes(tickvals=["MACO_b", "MACO_p=-10", "MACO_w=steep", "UF1", "UF2", "UF3", "ZDT1", "ZDT2", "ZDT3"],
                        ticktext=["MACO<br>b", "MACO<br>p=-10", "MACO<br>w=steep", "UF1", "UF2", "UF3", "ZDT1", "ZDT2", "ZDT3"])
    #fig.update_yaxes(tickvals=["MACO_b"], ticktext=["MACO \n b"])
    fig.update_layout(
        height=350,
        #width=150*len(filtered_df.problem_name.unique()),
        margin=dict(l=0, r=0, t=25, b=0),
        font=dict(size=13)
    )
    fig.write_image("../figures/paper/performance_comparison/roi_"+indicator_name+"_d"+str(dim)+".pdf")
    fig.show()

plot_heatmap(baseline_comparison_d2, "percentual_gd_diff", dim=2)
plot_heatmap(baseline_comparison_d3, "percentual_gd_diff", dim=3)

plot_heatmap(baseline_comparison_d2, "percentual_igd+_diff", dim=2)
plot_heatmap(baseline_comparison_d3, "percentual_igd+_diff", dim=3)

plot_heatmap(baseline_comparison_d2, "percentual_hv_diff", dim=2)
plot_heatmap(baseline_comparison_d3, "percentual_hv_diff", dim=3)



# Compare the PFs found

In [6]:
import pandas as pd
fitness_d2 = pd.read_csv("../data/e_and_c_benchmark/fitness_and_ranks_do2_gen100.csv.tar.gz")
fitness_d2 = fitness_d2.loc[(fitness_d2.problem_name != "ZDT4") & (fitness_d2.problem_name != "ZDT6") & (fitness_d2.problem_name != "MACO_w=shallow")]#filter unused problems
fitness_d2 = fitness_d2.loc[fitness_d2.goldberg_rank == 0]

fitness_d3 = pd.read_csv("../data/e_and_c_benchmark/fitness_and_ranks_do3_gen100.csv.tar.gz")
fitness_d3 = fitness_d3.loc[fitness_d3.goldberg_rank == 0]

In [7]:
# some helper methods

import plotly.graph_objects as go

def get_2d_scatterplot(data, seedID) -> go.scatter:
    return go.Scatter(
            x=data["f_1"],
            y=data["f_2"],
            mode="markers",
            marker=dict(
                color=data[seedID],
                colorscale=px.colors.sequential.algae,
                colorbar=dict(thickness=20, bordercolor='white', title="Impact"),
                cmin=0,
                cmax=1,
                opacity=0.5
            ),
            showlegend=False,
        )

def get_2d_pf_fig(problem) -> go.scatter:
    pf = problem._calc_pareto_front()
    return go.Scatter(
                x=pf[:, 0],
                y=pf[:, 1],
                mode="markers",
                marker=dict(size=3, color="grey"),
                showlegend=False
            )

In [9]:
import plotly.express as px

def plot_2d_data(data, problem_name):
    data = data[data.problem_name == problem_name]
    data = data.drop(columns=["problem_name"])

    fig = px.scatter(
        data,
        x="f_1",
        y="f_2",
        facet_col="seed_type",
        facet_row="algorithm_name",
        #color="seed_type",
        title=problem_name,
        labels={
            "traceID_1": "Impact"
        },
        facet_col_spacing=0.0,
        facet_row_spacing=0.0
    )


    #add PF plot to each sub plot
    for j in range(1, len(data['seed_type'].unique())+1):#rows
        for i in range(1, len(data['algorithm_name'].unique())+1):#cols
            fig.add_trace(get_2d_pf_fig(problems[problem_name]), row=i, col=j)
    fig.update_layout(title=problem_name, height=400, width=1100,
        margin=dict(l=0, r=0, t=25, b=0),
    )
    fig.for_each_annotation(lambda a: a.update( text=a.text.removeprefix("seed_type=") ))#remove the prefix
    fig.for_each_annotation(lambda a: a.update( text=a.text.removeprefix("algorithm_name=") ))#remove the prefix
    fig.write_image("../figures/test/pareto_fronts/single_seed_pf_"+problem_name+".pdf")


data = fitness_d2[["generation", "f_1", "f_2", "problem_name", "algorithm_name", "seed_type"]]
#data = data[(data.seed_type == "e1") | (data.seed_type == "e2") | (data.seed_type == "c")]
problem_names = data.problem_name.unique()
for problem_name in problem_names:
    plot_2d_data(data, problem_name)

In [10]:
import plotly.graph_objects as go

def get_3d_scatterplot(data, seedID) -> go.scatter3d:
    return go.Scatter3d(
            x=data["f_1"],
            y=data["f_2"],
            z=data["f_3"],
            mode="markers",
            showlegend=False,
        )

def get_3d_pf_fig(problem) -> go.scatter3d:
    pf = problem._calc_pareto_front()
    return go.Scatter3d(
                x=pf[:, 0],
                y=pf[:, 1],
                z=pf[:, 2],
                mode="markers",
                marker=dict(size=3, color="grey"),
                showlegend=False
            )


#The scene camera:
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1.5, y=1.5, z=1.5)
)

In [ ]:
# Plot all of the 3d one seed data
from test_setup import problems
from plotly.subplots import make_subplots

def plot_3d_data(data, problem_name, algorithm_name):
    data = data[data.algorithm_name == algorithm_name]
    data = data[data.problem_name == problem_name]
    data = data.drop(columns=["problem_name"])

    fig = make_subplots(
        rows=1,
        cols=7,
        subplot_titles=("r", "e1", "e2", "e3", "c", "e1+e2+e3", "e1+e2+e3+c"),
        start_cell="top-left",
        specs=[
            [{"type": "scatter3d"}, {"type": "scatter3d"}, {"type": "scatter3d"}, {"type": "scatter3d"}, {"type": "scatter3d"}, {"type": "scatter3d"}, {"type": "scatter3d"}],
        ],
        horizontal_spacing = 0.01
    )

    for i, seed_type in enumerate(data.seed_type.unique()): #columns
        fig.update_scenes(
            camera=camera,
            row=1, col=i+1
        )
        fig.add_trace(get_3d_scatterplot(data[data.seed_type == seed_type], "traceID_1"), row=1, col=i+1)
        fig.add_trace(get_3d_pf_fig(problems[problem_name]), row=1, col=i+1) #add the true PF
        
    fig.update_layout(title=problem_name+" "+algorithm_name, height=400, width=1600,
        margin=dict(l=0, r=0, t=35, b=0),
    )

    fig.write_image("../figures/test/pareto_fronts/single_seed_pf_"+problem_name+"_"+algorithm_name+".pdf")

data = fitness_d3[["generation", "f_1", "f_2", "f_3", "problem_name", "algorithm_name", "seed_type"]]
problem_names = data.problem_name.unique()
for problem_name in problem_names:
    plot_3d_data(data, problem_name, "NSGA2")
    plot_3d_data(data, problem_name, "MOEAD")


# Line Plot Average Performance

In [ ]:
import pandas as pd
baseline_comparison_d2 = pd.read_csv("../data/e_and_c_benchmark/baseline_comparison_data_d2.csv.tar.gz")
baseline_comparison_d3 = pd.read_csv("../data/e_and_c_benchmark/baseline_comparison_data_d3.csv.tar.gz")

baseline_comparison_d2

In [ ]:
import plotly.express as px

def line_plot_average(dataframe, performance_indicator, problem_family, log_y=True):
    fig = px.line(
        dataframe.loc[(dataframe.problem_name.str.contains(problem_family))],
        x="generation",
        y=performance_indicator,
        color="seed_type",
        facet_col="problem_name",
        facet_row="algorithm",
        title=performance_indicator+" "+problem_family,
        log_y=log_y
    )
    fig.show()

#igd+
# line_plot_average(baseline_comparison_d3, "average_igd+", "DTLZ")
# line_plot_average(baseline_comparison_d3, "average_igd+", "UF")
# line_plot_average(baseline_comparison_d2, "average_igd+", "ZDT")
# line_plot_average(baseline_comparison_d2, "average_igd+", "UF")
line_plot_average(baseline_comparison_d2, "average_igd+", "MACO")

#gd
# line_plot_average(baseline_comparison_d3, "average_gd", "DTLZ")
# line_plot_average(baseline_comparison_d3, "average_gd", "UF")
# line_plot_average(baseline_comparison_d2, "average_gd", "ZDT")
# line_plot_average(baseline_comparison_d2, "average_gd", "UF")
# line_plot_average(baseline_comparison_d2, "average_gd", "MACO")

# #hv
# line_plot_average(baseline_comparison_d3, "average_hv", "DTLZ")
# line_plot_average(baseline_comparison_d3, "average_hv", "UF")
# line_plot_average(baseline_comparison_d2, "average_hv", "ZDT")
# line_plot_average(baseline_comparison_d2, "average_hv", "UF")
# line_plot_average(baseline_comparison_d2, "average_hv", "MACO")


# Comparing Final Generation Performance

In [27]:
import pandas as pd
performance_indicators_d2 = pd.read_csv("../data/e_and_c_benchmark/performance_indicators_do2_plus_HV.csv")
performance_indicators_d2 = performance_indicators_d2.loc[performance_indicators_d2.generation == performance_indicators_d2.generation.max()]
performance_indicators_d3 = pd.read_csv("../data/e_and_c_benchmark/performance_indicators_do3_plus_HV.csv")
performance_indicators_d3 = performance_indicators_d3.loc[performance_indicators_d3.generation == performance_indicators_d3.generation.max()]

In [ ]:
import plotly.express as px

def line_plot_average(dataframe, performance_indicator, problem_family):
    fig = px.box(
        dataframe.loc[(dataframe.problem_name.str.contains(problem_family))],
        x="seed_type",
        y=performance_indicator,
        color="seed_type",
        facet_col="problem_name",
        facet_row="algorithm_name",
        title=performance_indicator+" "+problem_family,
        log_y=True
    )
    fig.show()

#igd+
# line_plot_average(performance_indicators_d3, "igd+", "DTLZ")
# line_plot_average(performance_indicators_d3, "igd+", "UF")
# line_plot_average(performance_indicators_d2, "igd+", "ZDT")
line_plot_average(performance_indicators_d2, "igd+", "UF")
line_plot_average(performance_indicators_d2, "igd+", "MACO")

#gd
# line_plot_average(performance_indicators_d3, "gd", "DTLZ")
# line_plot_average(performance_indicators_d3, "gd", "UF")
# line_plot_average(performance_indicators_d2, "gd", "ZDT")
# line_plot_average(performance_indicators_d2, "gd", "UF")
# line_plot_average(performance_indicators_d2, "gd", "MACO")

#hv
# line_plot_average(performance_indicators_d3, "hv", "DTLZ")
# line_plot_average(performance_indicators_d3, "hv", "UF")
# line_plot_average(performance_indicators_d2, "hv", "ZDT")
# line_plot_average(performance_indicators_d2, "hv_ref2", "UF")
# line_plot_average(performance_indicators_d2, "hv_ref1", "MACO")
# line_plot_average(performance_indicators_d2, "hv_ref2", "MACO")